### Rasberry Pi
   how to make a rasberry pi a cluster computer

- burn image via sd card
- Expand the File System If needed. 
- Change the hostname to Pi01
- Change the boot option to CLI
- "Interfaces" tab to make sure SSH is enabled. 
- "Overclock" tab to choose "Turbo". 
- Change the GPU memory to 16mb. 
- reboot your Pi.

### MPICH (Message Passing Interface)
	mkdir mpich
    wget http://www.mpich.org/static/downloads/3.3.1/mpich-3.3.1.tar.gz
	tar xfz mpich-3.3.1.tar.gz
	mkdir /home/litzen/mpi-install
    mkdir /home/litzen/mpi-build
    cd mpi-build
	sudo /home/litzen/mpich/mpich-3.3.1/configure -prefix=/home/litzen/mpi-install
	sudo make
	sudo make install
    
Failed on Win10  
https://www.microsoft.com/en-us/download/confirmation.aspx?id=54607  
PATH=%PATH%;C:\Program Files\Microsoft MPI\bin

### mpi4py
from source:
- wget https://bitbucket.org/mpi4py/mpi4py/downloads/mpi4py-3.0.2.tar.gz
- tar xfz mpi4py-3.0.2.tar.gz
- cd mpi4py-3.0.2
- python setup.py build
- python setup.py install

pip install:
- sudo apt install libopenmpi-dev
- pip install mpi4py	

In [3]:
# test in Linux
!mpiexec -n 1 hostname
!mpiexec -version

Litzen
mpiexec (OpenRTE) 2.1.1

Report bugs to http://www.open-mpi.org/community/help/


In [6]:
# helloworld.py
# initialize MPI enviroment
from mpi4py import MPI
# MPI.COMM_WORLD: Communicator Class
# comm: object of class
comm = MPI.COMM_WORLD
# .Get_size, .Get_rank: Method
size = comm.Get_size()
rank = comm.Get_rank()
name = MPI.Get_processor_name()

print("hello world! I'm processor %d of %d on %s \n" % (rank, size, name))

hello world! I'm processor 0 of 1 on Litzen 



In [1]:
!mpiexec -n 16 python helloworld.py
# Multiple-node
# !mpiexec -n 16 -host node1, node2, node3, python helloworld.py
# !mpiexec -n 16 -machinefile python helloworld.py

--------------------------------------------------------------------------
btl_vader_single_copy_mechanism MCA variable, but CMA support is
not available due to restrictive ptrace settings.

The vader shared memory BTL will fall back on another single-copy
mechanism if one is available. This may result in lower performance.

  Local host: Litzen
--------------------------------------------------------------------------
hello world! I'm processor 4 of 16 on Litzen 

hello world! I'm processor 5 of 16 on Litzen 

hello world! I'm processor 6 of 16 on Litzen 

hello world! I'm processor 9 of 16 on Litzen 

hello world! I'm processor 14 of 16 on Litzen 

hello world! I'm processor 15 of 16 on Litzen 

hello world! I'm processor 1 of 16 on Litzen 

hello world! I'm processor 8 of 16 on Litzen 

hello world! I'm processor 10 of 16 on Litzen 

hello world! I'm processor 11 of 16 on Litzen 

hello world! I'm processor 13 of 16 on Litzen 

hello world! I'm processor 0 of 16 on Litzen 

hello wo

For normal python objects (no buffer): 
- .send(), .recv(), .bcast(), .scatter(), .gather()  
    sent objective as a paramenter of the communication call  
    the received object is simply the return value.
- .isend() .irecv()  
     return an object of Request(class) instances    
     completed by test() and wait() methods of the Request class

In [17]:
#comm_p2p_blocking.py
# blocking p2p: 
#.send(obj=data, int dest, int tag=0) 
#.recv(int source=ANY, int tag=0) return received object
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # dict with {key: value}, set with {key}
    # Sequence: list with [], Tuple with ()
    data = {'a': 1, 'b': 2}
    print('process %d sends %s' %(rank, data))
    # can specify with 'tag'
    comm.send(data, dest=1)
elif rank == 1:
    data = comm.recv(source=0)
    print('process %d receives %s' %(rank, data))

process 0 sends {'a': 1, 'b': 2}


Exception: MPI_ERR_RANK: invalid rank

In [18]:
!mpiexec -n 2 python comm_p2p_blocking.py

--------------------------------------------------------------------------
btl_vader_single_copy_mechanism MCA variable, but CMA support is
not available due to restrictive ptrace settings.

The vader shared memory BTL will fall back on another single-copy
mechanism if one is available. This may result in lower performance.

  Local host: Litzen
--------------------------------------------------------------------------
process 0 sends {'a': 1, 'b': 2}
process 1 receives {'a': 1, 'b': 2}
[Litzen:00238] 1 more process has sent help message help-btl-vader.txt / cma-permission-denied
[Litzen:00238] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages


In [128]:
#comm_p2p_unblocking.py
# unblocking p2p: calculating while communicating
# .isend(obj, int dest, int tag=0) 
# .irecv(int source =ANY, int tag=ANY) 
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 1, 'b': 2}
    print('process %d sends %s' %(rank, data))
    # .isend begin to send and return a MPI.Request object
    req = comm.isend(data, dest=1)
    # .wait() wait for receive
    req.wait()
elif rank == 1:
    # . irecv begin to receive
    req = comm.irecv(source=0)
    # req.wait() returns received data or None
    data = req.wait()
    print('process %d receives %s' %(rank, data))

process 0 sends {'a': 1, 'b': 2}


Exception: MPI_ERR_RANK: invalid rank

In [1]:
!mpiexec -n 2 python comm_p2p_unblocking.py

--------------------------------------------------------------------------
btl_vader_single_copy_mechanism MCA variable, but CMA support is
not available due to restrictive ptrace settings.

The vader shared memory BTL will fall back on another single-copy
mechanism if one is available. This may result in lower performance.

  Local host: Litzen
--------------------------------------------------------------------------
process 0 sends {'a': 1, 'b': 2}
process 1 receives {'a': 1, 'b': 2}
[Litzen:00055] 1 more process has sent help message help-btl-vader.txt / cma-permission-denied
[Litzen:00055] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages


In [45]:
# comm_bcast.py
#.bcast() for normal python objects

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'black', 'white'}
    print('before bcast: process %d has %s' %(rank, data))
else:
    data = None
    print('before bcast: process %d has %s' %(rank, data))
# need object 'data' receive .bcast() on each process
data = comm.bcast(data, root=0)
print('after bcast: process %d has %s' %(rank, data))

before bcast: process 0 has {'black', 'white'}
after bcast: process 0 has {'black', 'white'}


In [2]:
!mpiexec -n 4 python comm_bcast.py

--------------------------------------------------------------------------
btl_vader_single_copy_mechanism MCA variable, but CMA support is
not available due to restrictive ptrace settings.

The vader shared memory BTL will fall back on another single-copy
mechanism if one is available. This may result in lower performance.

  Local host: Litzen
--------------------------------------------------------------------------
before bcast: process 2 has None
before bcast: process 1 has None
before bcast: process 0 has {'white', 'black'}
before bcast: process 3 has None
after bcast: process 0 has {'white', 'black'}
after bcast: process 1 has {'white', 'black'}
after bcast: process 2 has {'white', 'black'}
after bcast: process 3 has {'white', 'black'}
[Litzen:00067] 3 more processes have sent help message help-btl-vader.txt / cma-permission-denied
[Litzen:00067] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages


In [80]:
# comm_scatter.py
#.scatter() for normal python objects

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    sdata = range(size)
    print('before scatter: process %d has %s' %(rank, sdata))
else:
    #sdata = None
    print('before scatter: process %d has %s' %(rank, sdata))
# need object 'data' receive .bcast()
rdata = comm.scatter(sdata, root=0)
print('after bcast: process %d has %s' %(rank, rdata))

before scatter: process 0 has range(0, 1)
after bcast: process 0 has 0


In [ ]:
!mpiexec -n 3 python comm_scatter.py

In [117]:
# comm_gather.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

data = rank
print('before gathering: process %d has %s' % (rank, data))

data = comm.gather(data, root=0)
print('after scattering: process %d has %s' % (rank, data))

before gathering: process 0 has 0
after scattering: process 0 has [0]


In [ ]:
!mpiexec -n 3 python comm_gather.py

For Numpy Arrays (buffer):
- .Send(), .Recv(), .Bcast(), .Scatter, .Gather()  
    specify arguments

In [34]:
# comm_p2p_numpy.py
# .Send(list buf=[data, count, MPI.INT], int dest, int tag=0)
# .Recv(list buf=[data, count, MPI.INI], int source=ANY, int tag=ANY)
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

import numpy as np

if rank==0:
    data = np.arange(10, dtype='i')
    # specify data type by sending a list (or automatically specified) 
    # .Send() must specify 'tag'
    comm.Send([data, MPI.INT], dest=1, tag=1)
    print('process %d sends %s' %(rank, data))
elif rank==1:
    data = np.empty(10, dtype='i')
    #receive and store in data
    comm.Recv([data, MPI.INT], source=0, tag=1)
    print('process %d receive %s' %(rank, data))

Exception: MPI_ERR_RANK: invalid rank

In [47]:
!mpiexec -n 2 python comm_p2p_numpy.py

--------------------------------------------------------------------------
btl_vader_single_copy_mechanism MCA variable, but CMA support is
not available due to restrictive ptrace settings.

The vader shared memory BTL will fall back on another single-copy
mechanism if one is available. This may result in lower performance.

  Local host: Litzen
--------------------------------------------------------------------------
process 0 sends [0 1 2 3 4 5 6 7 8 9]
process 1 receive [0 1 2 3 4 5 6 7 8 9]
[Litzen:00408] 1 more process has sent help message help-btl-vader.txt / cma-permission-denied
[Litzen:00408] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages


In [68]:
# comm_bcast_numpy.py

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

import numpy as np

if rank == 0:
    data = np.arange(10, dtype='i')
    print('before bcast: process %d has %s' %(rank, data))
else:
    data = np.zeros(10, dtype='i')
    print('before bcast: process %d has %s' %(rank, data))

comm.Bcast(data, root=0)
print('after bcast: process %d has %s' %(rank, data))

before bcast: process 0 has [0 1 2 3 4 5 6 7 8 9]
after bcast: process 0 has [0 1 2 3 4 5 6 7 8 9]


In [72]:
!mpiexec -n 4 python comm_bcast_numpy.py

--------------------------------------------------------------------------
btl_vader_single_copy_mechanism MCA variable, but CMA support is
not available due to restrictive ptrace settings.

The vader shared memory BTL will fall back on another single-copy
mechanism if one is available. This may result in lower performance.

  Local host: Litzen
--------------------------------------------------------------------------
before bcast: process 0 has [0 1 2 3 4 5 6 7 8 9]
before bcast: process 1 has [0 0 0 0 0 0 0 0 0 0]
after bcast: process 0 has [0 1 2 3 4 5 6 7 8 9]
after bcast: process 1 has [0 1 2 3 4 5 6 7 8 9]
before bcast: process 2 has [0 0 0 0 0 0 0 0 0 0]
after bcast: process 2 has [0 1 2 3 4 5 6 7 8 9]
before bcast: process 3 has [0 0 0 0 0 0 0 0 0 0]
after bcast: process 3 has [0 1 2 3 4 5 6 7 8 9]
[Litzen:00666] 3 more processes have sent help message help-btl-vader.txt / cma-permission-denied
[Litzen:00666] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / 

In [114]:
# comm_scatter_numpy.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
#on each process
sdata = None
if rank == 0:
    sdata = np.random.normal(0, 1, size=(size, 3))
    
# on each process:
print('before scatter: process %d has\n%s' %(rank, sdata))
rdata = np.zeros(3)
comm.Scatter(sdata, rdata, root=0)
print('after bcast: process %d has %s' %(rank, rdata))

before scatter: process 0 has
[[1.13547712 0.25592499 2.88306616]]
after bcast: process 0 has [1.13547712 0.25592499 2.88306616]


In [115]:
!mpiexec -n 3 python comm_scatter_numpy.py

--------------------------------------------------------------------------
btl_vader_single_copy_mechanism MCA variable, but CMA support is
not available due to restrictive ptrace settings.

The vader shared memory BTL will fall back on another single-copy
mechanism if one is available. This may result in lower performance.

  Local host: Litzen
--------------------------------------------------------------------------
before scatter: process 0 has
[[-0.22474025  0.03019099  1.95457882]
 [-0.5791027  -0.71683518  0.20883896]
 [ 1.23454389 -0.60295366  0.95887501]]
after bcast: process 0 has [-0.22474025  0.03019099  1.95457882]
before scatter: process 1 has
None
before scatter: process 2 has
None
after bcast: process 1 has [-0.5791027  -0.71683518  0.20883896]
after bcast: process 2 has [ 1.23454389 -0.60295366  0.95887501]
[Litzen:00984] 2 more processes have sent help message help-btl-vader.txt / cma-permission-denied
[Litzen:00984] Set MCA parameter "orte_base_help_aggregate" to 0 t

In [125]:
# comm_gather_numpy.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

sdata = np.zeros(3) + rank
print('before gathering: process %d has %s' %(rank, sdata))

rdata = None
if rank == 0:
    rdata = np.zeros([size, 3])

comm.Gather(sdata, rdata, root=0)
print('after gathering, process %d has %s' %(rank, rdata))

before gathering: process 0 has [0. 0. 0.]
after gathering, process 0 has [[0. 0. 0.]]


In [126]:
!mpiexec -n 4 python comm_gather_numpy.py

--------------------------------------------------------------------------
btl_vader_single_copy_mechanism MCA variable, but CMA support is
not available due to restrictive ptrace settings.

The vader shared memory BTL will fall back on another single-copy
mechanism if one is available. This may result in lower performance.

  Local host: Litzen
--------------------------------------------------------------------------
before gathering: process 3 has [3. 3. 3.]
after gathering, process 3 has
None
before gathering: process 2 has [2. 2. 2.]
after gathering, process 2 has
None
before gathering: process 1 has [1. 1. 1.]
after gathering, process 1 has
None
before gathering: process 0 has [0. 0. 0.]
after gathering, process 0 has
[[0. 0. 0.]
 [1. 1. 1.]
 [2. 2. 2.]
 [3. 3. 3.]]
[Litzen:01038] 3 more processes have sent help message help-btl-vader.txt / cma-permission-denied
[Litzen:01038] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages
